In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import skimage
from skimage.transform import resize
from skimage.io import imread, imshow
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, auc

from sklearn import svm

import cv2

In [79]:
# Setting input directory for bird and squirrel images

input_dir = '/content/drive/MyDrive/CV_data'

# listing possible image categories in folder
categories = ['Bird', 'Squirrel']

In [80]:

def prepare_images(input_dir, category):

'''
This function takes in the input directory and the category of the images and returns the images and their labels as a list.
The function also resizes the images to 224x224 pixels and converts the image to grayscale.

Args:

input_dir: str: The directory where the images are stored
category: str: The category of the images/labels of the folders that contain the images

Returns:

data_array: list: A list containing flattened image information

labels_array: list: A list of the labels of the images
'''

# creating empty lists to store data and labels
data = []
labels = []

    # looping through Bird and Squirrel categories in the folder

for category in categories:

    # looping through the images in each category

    for file in os.listdir(os.path.join(input_dir, category)):

        # reading the image

        image = imread(os.path.join(input_dir, category, file))

        # resizing the image

        image = resize(image, (224, 224, 3))

        #converting the image to greyscale

        image = skimage.color.rgb2gray(image)

        # normalizing the pixel values

        image = image / 255

        # flattening the image

        image = image.flatten()

        # append the image data to the list data

        data.append(image)

        # append the label to the list labels

        labels.append(category)

# converting lists data and labels into numpy arrays
data_array = np.array(data)
labels_array = np.array(labels)

# returning the completed arrays
return data_array, labels_array

SVM Model Performance

In [81]:
def train_svm(data_array, labels_array):

  '''
  This function takes in the data and labels arrays and splits them into training and testing sets using sklearn's train_test_split
  20 percent of the data is used for testing and 80 percent for training
  The function then trains a support vector machine model using the training data and tests the model using the testing data
  The function then prints the accuracy, confusion matrix and classification report of the model
  The function returns the trained model

  Args:

  data_array: numpy array of the data
  labels_array: numpy array of the labels

  Returns: svm_model: trained support vector machine model
  '''

# using train test split to split data into training and test sets

X_train, X_test, y_train, y_test = train_test_split(data_array, labels_array, test_size=0.2, random_state=42)

# initializing the SVM model

svm_model = svm.SVC(kernel='linear', C=1, gamma=1)

# training the SVM model
svm_model.fit(X_train, y_train)

# making predictions on the test set
y_pred = svm_model.predict(X_test)

# printing accuracy, confusion matrix, and classification report

print("Accuracy:", round(accuracy_score(y_test, y_pred), 2))

print(confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred))

# returning trained svm_model
return svm_model

In [82]:
# preparing data using the prepare_images function above

data_array, labels_array = prepare_images(input_dir, categories)

# training the SVC model and printing results
svm_model = train_svm(data_array, labels_array)

Accuracy: 0.61
[[1017    1]
 [ 664   38]]
              precision    recall  f1-score   support

        Bird       0.60      1.00      0.75      1018
    Squirrel       0.97      0.05      0.10       702

    accuracy                           0.61      1720
   macro avg       0.79      0.53      0.43      1720
weighted avg       0.76      0.61      0.49      1720



In [ ]:
# Ouput Directory
output_dir = '/content/Test_Images'


# Saving Test Set Images for Use in Mean Model Comparison
for i in range(0, len(X_test)):

    plt.imshow(X_test[i].reshape(224,224), cmap='gray')
    plt.savefig(os.path.join(output_dir, f'Test_{i}.png'))
    plt.close()

Mean Model Comparison


In [83]:
# importing required packages
import torch
from torchvision import models
from torchvision import transforms
from torchvision.io import read_image, ImageReadMode

In [91]:
def alex_net_model(X_test):

  '''

  This function calls a pretrained AlexNet model
  It runs the test images we created above through the network and generates predictions
  It then prints these classification predictions along with the probability percentage

  Args:

  X_test: numpy array of the test images

  Returns: None

  '''

  # initializing pretrained alexnet model
  model = models.alexnet(pretrained=True)
  # putting model in evaluation mode
  model.eval()

  # defining transforms that will be applied to the image

  preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
  )])

  # initializing empty list for storing predictions
  preds = []

  # calling classes used in alexnet model for labelling
  with open('/content/Doc4.txt') as f:
    classes = [line.strip() for line in f.readlines()]

    # iterate through images in Test_Images folder, transforming images, conerting to tensor, then
    # running through alexnet network and printing list of predictions along with image label

  for i in range(0, len(X_test)):

    test_image_path = f'/content/Test_Images/Test_{i}.png'

    img = Image.open(test_image_path).convert('RGB')

    img_resized = img.resize((256,256))

    img_t = preprocess(img_resized)

    batch_t = torch.unsqueeze(img_t, 0)

    out = alexnet(batch_t)

    _ , indices = torch.sort(out, descending = True)

    percentage = torch.nn.functional.softmax(out, dim = 1)[0]*100

    print([(f'Test_{i}.png', classes[idx], percentage[idx].item()) for idx in indices[0][:1]])


In [92]:
alex_net_model(X_test)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[('Test_0.png', "163: 'bloodhound, sleuthhound',", 17.798553466796875)]
[('Test_1.png', "24: 'great grey owl, great gray owl, Strix nebulosa',", 11.786393165588379)]
[('Test_2.png', "456: 'bow',", 8.806989669799805)]
[('Test_3.png', "702: 'parallel bars, bars',", 8.67601203918457)]
[('Test_4.png', "19: 'chickadee',", 19.416351318359375)]
[('Test_5.png', "749: 'quill, quill pen',", 14.097225189208984)]
[('Test_6.png', "445: 'bikini, two-piece',", 29.146108627319336)]
[('Test_7.png', "164: 'bluetick',", 11.506406784057617)]
[('Test_8.png', "677: 'nail',", 9.151708602905273)]
[('Test_9.png', "512: 'corkscrew, bottle screw',", 10.034878730773926)]
[('Test_10.png', "445: 'bikini, two-piece',", 12.821274757385254)]
[('Test_11.png', "285: 'Egyptian cat',", 15.141386985778809)]
[('Test_12.png', "921: 'book jacket, dust cover, dust jacket, dust wrapper',", 54.79890060424805)]
[('Test_13.png', "405: 'airship, dirigible',", 10.203818321228027)]
[('Test_14.png', "82: 'ruffed grouse, partridge, Bon

In [ ]:
# Accuracy 24%